In [1]:
"""
PROJECT: NeuralTranscript: Semantic Search & Q&A for YouTube Content
MODULE: 02_SEMANTIC_CHUNKING
-------------------------------------------------------------------------
DESCRIPTION:
This module transforms raw transcript text into semantically meaningful 
chunks. By adding source metadata to each chunk, we enable the RAG system 
to provide citations and structured context to the LLM (Gemini/Groq).

AUTHOR: Engr. Inam Ullah Khan
Master's Student in Data Science | Al-Farabi Kazakh National University
-------------------------------------------------------------------------
"""

import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

# --- 1. CONFIGURATION ---
VIDEO_ID = "Gfr50f6ZBvo"
INPUT_PATH = f"data/transcripts/{VIDEO_ID}.txt"

# RAG Hyperparameters (Based on your thesis research)
CHUNK_SIZE = 1000   # Max characters per chunk
CHUNK_OVERLAP = 200 # Context window between chunks to prevent data loss

# --- 2. CORE PROCESSING FUNCTIONS ---

def load_processed_transcript(file_path: str) -> str:
    """Loads the unified transcript text from Notebook 01."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ Transcript not found at {file_path}. Run Notebook 01 first.")
    
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

def create_enriched_chunks(text: str, source_id: str) -> list[Document]:
    """
    Splits text into chunks and attaches metadata for agentic retrieval.
    
    Args:
        text (str): The full transcript text.
        source_id (str): The Video ID for citation purposes.
        
    Returns:
        list[Document]: A list of LangChain Document objects with metadata.
    """
    print(f"✂️ Initializing Recursive Splitting (Size: {CHUNK_SIZE}, Overlap: {CHUNK_OVERLAP})...")
    
    # Initialize the splitter
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=["\n\n", "\n", ".", " ", ""], # Hierarchy of split points
        add_start_index=True # Tracks exactly where the chunk starts in the original text
    )
    
    # Generate raw text chunks
    raw_chunks = splitter.split_text(text)
    
    # Wrap in Document objects with enriched metadata
    enriched_docs = [
        Document(
            page_content=chunk,
            metadata={
                "source": source_id,
                "chunk_id": i,
                "content_type": "video_transcript",
                "char_length": len(chunk)
            }
        )
        for i, chunk in enumerate(raw_chunks)
    ]
    
    return enriched_docs

# --- 3. EXECUTION PIPELINE ---

if __name__ == "__main__":
    print(f"--- Starting NeuralTranscript Chunking Pipeline ---")
    
    # Step 1: Load Data
    full_text = load_processed_transcript(INPUT_PATH)
    
    # Step 2: Semantic Chunking
    chunked_docs = create_enriched_chunks(full_text, VIDEO_ID)
    
    # Step 3: Analysis & Validation
    total_chunks = len(chunked_docs)
    print(f"✅ Created {total_chunks} enriched chunks.")
    
    # Verify the first chunk structure
    sample = chunked_docs[0]
    print("\n--- CHUNK STRUCTURE VALIDATION ---")
    print(f"Metadata: {sample.metadata}")
    print(f"Preview: {sample.page_content[:150]}...")

    # For your Portfolio: Save a summary of chunks for visual confirmation
    print(f"\n🚀 Ready for Notebook 03: Vector Embedding & FAISS Indexing")

d:\AgenticRAG\LangChain-Transcript-QA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'langchain.schema'

In [ ]:
"""
PROJECT: NeuralTranscript: Semantic Search & Q&A for YouTube Content
MODULE: 02_SEMANTIC_CHUNKING
-------------------------------------------------------------------------
DESCRIPTION:
This module transforms raw transcript text into semantically meaningful 
chunks. By adding source metadata to each chunk, we enable the RAG system 
to provide citations and structured context to the LLM (Gemini/Groq).

AUTHOR: Engr. Inam Ullah Khan
Master's Student in Data Science | Al-Farabi Kazakh National University
-------------------------------------------------------------------------
"""

import os
# NEW: Import from the dedicated text-splitters package
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# --- 1. CONFIGURATION ---
VIDEO_ID = "Gfr50f6ZBvo"
INPUT_PATH = f"data/transcripts/{VIDEO_ID}.txt"

# RAG Hyperparameters
CHUNK_SIZE = 1000   
CHUNK_OVERLAP = 200 

# --- 2. CORE PROCESSING FUNCTIONS ---

def load_processed_transcript(file_path: str) -> str:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ Transcript not found at {file_path}. Run Notebook 01 first.")
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

def create_enriched_chunks(text: str, source_id: str) -> list[Document]:
    print(f"✂️ Initializing Recursive Splitting (Size: {CHUNK_SIZE}, Overlap: {CHUNK_OVERLAP})...")
    
    # Updated Splitter
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=["\n\n", "\n", ".", " ", ""],
        add_start_index=True 
    )
    
    # Generate chunks as Document objects
    # Note: Using create_documents is cleaner in the new API
    enriched_docs = splitter.create_documents(
        [text], 
        metadatas=[{"source": source_id, "content_type": "video_transcript"}]
    )
    
    return enriched_docs

# --- 3. EXECUTION PIPELINE ---

if __name__ == "__main__":
    print(f"--- Starting NeuralTranscript Chunking Pipeline ---")
    
    full_text = load_processed_transcript(INPUT_PATH)
    chunked_docs = create_enriched_chunks(full_text, VIDEO_ID)
    
    print(f"✅ Created {len(chunked_docs)} enriched chunks.")
    
    # Preview
    sample = chunked_docs[0]
    print(f"\n--- CHUNK VALIDATION ---\nMetadata: {sample.metadata}\nPreview: {sample.page_content[:150]}...")

--- Starting NeuralTranscript Chunking Pipeline ---
✂️ Initializing Recursive Splitting (Size: 1000, Overlap: 200)...
✅ Created 169 enriched chunks.

--- CHUNK VALIDATION ---
Metadata: {'source': 'Gfr50f6ZBvo', 'content_type': 'video_transcript', 'start_index': 0}
Preview: the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible ...


# Text Chunking Analysis

This notebook analyzes **text chunking strategies** for long-form unstructured text.
The goal is to identify suitable chunk sizes and overlaps for downstream
embedding and retrieval in a RAG pipeline.


In [ ]:

import os
from typing import List

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document


In [ ]:
# Path to transcript created in 01_data_ingestion.ipynb
DATA_PATH = "../data/transcripts"

# Replace with your video ID
VIDEO_ID = "VIDEO_ID_HERE"

file_path = os.path.join(DATA_PATH, f"{VIDEO_ID}.txt")

with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

print("Transcript loaded.")
print(f"Total characters: {len(text)}")


In [ ]:
# Convert raw text into a LangChain Document
documents = [Document(page_content=text)]


In [ ]:
# Different chunking strategies to compare
chunk_configs = [
    {"chunk_size": 500, "chunk_overlap": 50},
    {"chunk_size": 800, "chunk_overlap": 100},
    {"chunk_size": 1000, "chunk_overlap": 150},
]


In [ ]:
def chunk_documents(
    docs: List[Document],
    chunk_size: int,
    chunk_overlap: int
) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(docs)


In [ ]:
chunked_results = {}

for config in chunk_configs:
    chunks = chunk_documents(
        documents,
        chunk_size=config["chunk_size"],
        chunk_overlap=config["chunk_overlap"]
    )
    key = f'size_{config["chunk_size"]}_overlap_{config["chunk_overlap"]}'
    chunked_results[key] = chunks

    print(f"\nConfiguration: {key}")
    print(f"Number of chunks: {len(chunks)}")


In [ ]:
# Inspect first chunk from each configuration
for key, chunks in chunked_results.items():
    print("\n" + "="*60)
    print(f"Configuration: {key}")
    print("Chunk length:", len(chunks[0].page_content))
    print("Sample chunk preview:\n")
    print(chunks[0].page_content[:500])


## Observations

- Smaller chunks (500 characters) produce more granular segments but risk
  losing broader context.
- Larger chunks (1000 characters) preserve context but reduce retrieval precision.
- A chunk size of **~800 characters with overlap ~100** provides a good balance
  between semantic completeness and retrievability.

**Selected configuration for downstream RAG pipeline:**  
`chunk_size = 800`, `chunk_overlap = 100`


In [ ]:
# Save selected chunks for reuse
SELECTED_KEY = "size_800_overlap_100"
selected_chunks = chunked_results[SELECTED_KEY]

os.makedirs("../data/chunks", exist_ok=True)

output_path = f"../data/chunks/{VIDEO_ID}_chunks.txt"

with open(output_path, "w", encoding="utf-8") as f:
    for i, chunk in enumerate(selected_chunks):
        f.write(f"--- Chunk {i+1} ---\n")
        f.write(chunk.page_content + "\n\n")

print(f"Chunks saved to: {output_path}")


## Summary

- Evaluated multiple chunking strategies for long-form transcript data
- Conducted both quantitative and qualitative analysis
- Selected an optimal chunk configuration for embedding and retrieval

**Next step:** Embedding generation and similarity-based retrieval  
(`03_embedding_retrieval.ipynb`)
